In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from PIL import Image

# dataDir = './data/000/a01-003u.png'
dataDir = './data/000/a01-003u.png'
dirBase = './data'

# Pré-processamento

In [2]:
def augmentDataset(img, folder, nameImage, train, box):
    # Split do diretório atual para criação de uma nova base
    trainTestFolder = "Train" if train else "Test"

    # A partir das boxes criadas 
    for b in range (len(box)):
        (x, y, w, h) = box[b]
        
        path = "./data_augmented/" + folder + "/"
        pathDiscarted = "./data_augmented_discarted/" + folder + "/"

        imagePath = str(b) + "-"  + nameImage
        imagePathDiscarted = str(b) +  "-" + nameImage

        if not os.path.exists(pathDiscarted):
            os.makedirs(pathDiscarted)
            
        if img[y:y+h, x:x+w].shape[0] < 30 or img[y:y+h, x:x+w].shape[1] < 1000:

            cv2.imwrite(pathDiscarted + "/" + imagePathDiscarted, img[y:y+h, x:x+w])
        
        else:
            if not os.path.exists(path):
                # print("Diretório não existe")
                os.makedirs(path)

            trainTestDir = path + "/" + trainTestFolder + "/"
            if not os.path.exists(trainTestDir):
                os.makedirs(trainTestDir)

            cv2.imwrite(trainTestDir + imagePath, img[y:y+h, x:x+w])


def preProcess(img, dir, nameImage, train = True):

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY_INV)

    # Aplica o processo chamado de dilation
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (200,3))
    mask = cv2.morphologyEx(thresh, cv2.MORPH_DILATE, kernel)

    box = []
    contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]

    imgBoundingBox = img.copy()
    for cntr in contours:
        x,y,w,h = cv2.boundingRect(cntr)
        box.append((x,y,w,h))

    augmentDataset(imgBoundingBox, dir, nameImage, train, box)

In [3]:
limit = 0

files = os.listdir(dirBase)
limit = len(files) if limit == 0 else limit

for i, dir in enumerate(files[:limit]):
    imagesDir = os.listdir(f'{dirBase}/{dir}/')
    # print(dir, imagesDir[0])

    ImageDefaultTreino = cv2.imread(str(f'{dirBase}/{dir}/{imagesDir[0]}'))
    preProcess(ImageDefaultTreino, dir, imagesDir[0], True)

    ImageDefaultTeste = cv2.imread(str(f'{dirBase}/{dir}/{imagesDir[1]}'))
    preProcess(ImageDefaultTeste, dir, imagesDir[1], False)
    